In [1]:
import tensorflow as tf

# You'll generate plots of attention in order to see which parts of an image
# your model focuses on during captioning
import matplotlib.pyplot as plt

import collections
import random
import numpy as np
import os
import time
import json
from PIL import Image

In [14]:
random.seed(30)

In [3]:
# Download caption annotation files
annotation_folder = '/annotations/'
if not os.path.exists(os.path.abspath('.') + annotation_folder):
  annotation_zip = tf.keras.utils.get_file('captions.zip',
                                           cache_subdir=os.path.abspath('.'),
                                           origin='http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                           extract=True)
  annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2014.json'
  os.remove(annotation_zip)

# Download image files
image_folder = '/train2014/'
if not os.path.exists(os.path.abspath('.') + image_folder):
  image_zip = tf.keras.utils.get_file('train2014.zip',
                                      cache_subdir=os.path.abspath('.'),
                                      origin='http://images.cocodataset.org/zips/train2014.zip',
                                      extract=True)
  PATH = os.path.dirname(image_zip) + image_folder
  os.remove(image_zip)
else:
  PATH = os.path.abspath('.') + image_folder

13510582272/13510573713 [==============================] - 870s 0us/step


In [7]:
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

In [8]:
# Group all captions together having the same image ID.
image_path_to_caption = collections.defaultdict(list)
for val in annotations['annotations']:
  caption = f"<start> {val['caption']} <end>"
  image_path = PATH + 'COCO_train2014_' + '%012d.jpg' % (val['image_id'])
  image_path_to_caption[image_path].append(caption)

In [23]:
image_paths = list(image_path_to_caption.keys())
random.seed(30)
random.shuffle(image_paths)

train_image_paths = image_paths[:6000]
print(len(train_image_paths))

#print(train_image_paths[:3])

6000


In [25]:
train_captions = []
img_name_vector = []

for image_path in train_image_paths:
  caption_list = image_path_to_caption[image_path]
  train_captions.extend(caption_list)
  img_name_vector.extend([image_path] * len(caption_list))

print(len(img_name_vector))
print(len(train_captions))

30015
30015


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [32]:
!tar -xf /content/drive/MyDrive/Colab\ Notebooks/datasets/dataset.tar.gz

In [33]:
import glob

print(len(glob.glob("graduation/*.txt")))
train_captions2 = []
img_name_vector2 = []

for image_path in glob.glob("graduation/*.jpg"):
  caption_list = image_path_to_caption[image_path]
  with open(image_path.replace(".jpg","2.txt")) as f:
    #train_captions2.extend(caption_list)
    train_captions2.append("<start> " + f.read() + " <end>")
  img_name_vector2.extend([image_path])
train_captions.extend(train_captions2)
img_name_vector.extend(img_name_vector2)


6993


In [39]:
dataset_all = list(zip(img_name_vector,train_captions))

In [40]:
print(len(img_name_vector))
print(len(train_captions))
print(img_name_vector[-10:])
print(dataset_all[-10:])


37007
37007
['graduation/4673473841_789c058db9.jpg', 'graduation/10100422_0feca68bfb.jpg', 'graduation/149512183_05e8e86ec0.jpg', 'graduation/2851620331_119a6abd43.jpg', 'graduation/8967268358_4e00e61e2d.jpg', 'graduation/4721399529_5745aef56e.jpg', 'graduation/18497101025_b15a526a30.jpg', 'graduation/340741625_9c5b796bc1.jpg', 'graduation/g012_0102.jpg', 'graduation/5932422042_99b3ed82a2.jpg']
[('graduation/4673473841_789c058db9.jpg', '<start> a group of girls in graduation caps posing for a picture <end>'), ('graduation/10100422_0feca68bfb.jpg', '<start> a man and a woman standing on a wall <end>'), ('graduation/149512183_05e8e86ec0.jpg', '<start> a crowd of people watching a performance on a stage <end>'), ('graduation/2851620331_119a6abd43.jpg', '<start> a shark in the water with its mouth open <end>'), ('graduation/8967268358_4e00e61e2d.jpg', '<start> two women posing for a picture in front of the parliament building <end>'), ('graduation/4721399529_5745aef56e.jpg', '<start> a wom

In [41]:
random.seed(30)
random.shuffle(dataset_all)
train_vector = dataset_all[0:27755]
test_vector = dataset_all[27756:33306]
validation_vector = dataset_all[33307:37007]



In [47]:
import json

with open("dataset.json","w+") as f:
  f.write(json.dumps({
      "train_vector":train_vector,
      "test_vector":test_vector,
      "validation_vector":validation_vector
  }))

In [45]:
!du -sh train2014

13G	train2014
